## Authoritarian Framework

In [177]:
from dotenv import load_dotenv
load_dotenv()

True

In [178]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langgraph.checkpoint.sqlite import SqliteSaver

from langgraph.graph import END, StateGraph, MessageGraph

from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

import functools
import operator
from typing import List, Sequence, TypedDict, Annotated
import json
import os
import random

from IPython.display import Image, display

import concurrent.futures

In [179]:
unique_id = "Authoritarian Optimisation"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"

In [180]:
# from langsmith import Client

# client = Client()

In [204]:
class PromptReview(BaseModel):
    """Review of the prompt"""
    feedback: str = Field(..., description="Feedback on the most recent prompt")

class ApprovalDecision(BaseModel):
    """Decision on the approval of the prompt"""
    decision: bool = Field(..., description="Decision on if the prompt is good enough")

class CorePrinciples:
    def __init__(self, core_principles: List[str]):
        self.core_principles = core_principles
    
    def add_principle(self, principle: str):
        """
        Adds a principle to the core principles list.
        
        :param principle: The principle to be added.
        """
        self.core_principles.append(principle)
        
    def __str__(self):
        """
        Returns a string representation of the core principles, each principle is listed on a new line with a preceding dash.
        
        Example:
        - principle 1
        - principle 2
        ...
        """
        return "\n".join([f"- {principle}" for principle in self.core_principles])

class AdvisorAgent:
    """
    Advisor Agent class defining agents that provide feedback on prompts.
    """

    def __init__(self, position: str, core_principles: CorePrinciples, llm = ChatOpenAI(temperature=0.0, model="gpt-4o")):
        self.position = position
        self.core_principles = str(core_principles)
        self.system_message = SystemMessage(content=f"""You are an experienced: {self.position}. Your core principles are:
{self.core_principles}""")
        self.llm = llm

    def review_prompt(self, prompt: str, criteria: str) -> PromptReview:
        """
        Generates a review of the prompt.
        """
        template = """```{prompt}```

Your task is to think outside the box and provide feedback on the prompt above with creative recommendations on how to improve it in light of your core principles.

The success criteria for the updated prompt are as follows:
{criteria}
You must use this information to inform your feedback.

Your feedback must be actionable and less than 100 words. 
Provide clear and concise instructions on how to implement your recommendations.

Below are strict guidelines that you MUST follow when providing feedback:
- DO NOT suggest modifying existing restrictions.
- DO NOT suggest modifying or removing negations.
- DO NOT suggest adding, modifying or removing placeholders denoted by curly braces. IT IS ESSENTIAL PLACEHOLDERS DENOTED BY CURLY BRACES REMAIN IN PLACE.
- ALWAYS treat placeholders as the actual content.
You will be penalized if you do not follow these guidelines.

Your reviewal process should be as follows:
1. Read the prompt carefully as an expert {position}. 
2. Identify the most critical aspects of the prompt that need improvement. 
3. Think outside the box to provide creative feedback with recommendations on how to improve the prompt in light of your core principles.
4. Ensure that your feedback is actionable and less than 100 words.
5. Ensure that your feedback follows the strict guidelines provided above. DO NOT SUGGEST REMOVING PLACEHOLDERS.
6. Submit your feedback.

{format_instructions}

Output ONLY your feedback. DO NOT output any additional information.
Example JSON output: {{"feedback": "Your feedback here"}}

You will be penalized if your output cannot be parsed correctly."""
        pydantic_parser = PydanticOutputParser(pydantic_object=PromptReview)
        prompt_template = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["position", "prompt", "criteria"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt_template | self.llm | pydantic_parser
        for _ in range(3):
            try:
                completion = chain.invoke({"position": self.position, "prompt": prompt, "criteria": criteria})
                # Validate the output before returning
                if completion.feedback:
                    return completion
                # else:
                #     print("Validation failed: Missing required fields in completion")
                #     print("Raw output:", completion)
            except Exception as e:
                print("Exception occurred:", e)
                continue
        else:
            raise Exception("Failed to parse output after 3 attempts")
        
    def approval(self, prompt: str, criteria: str):
        """
        Agent to approve or reject the prompt.
        """
        template = """```{prompt}```
        
Your task is to review the prompt above and decide if it needs improvement in light of your core principles.

The success criteria for the prompt are as follows:
{criteria}
You must use this information to inform your decision.

If you think the prompt does not need improvements in light of your core principles and sufficiently meets the success criteria, return True. Otherwise, return False.

Your reviewal process should be as follows:
1. Read the prompt carefully as an expert {position}.
2. Determine whether the prompt needs improvement or meets the success criteria.
3. Submit your decision.

{format_instructions}

Output ONLY your decision. DO NOT output any additional information.
Example JSON output: {{"decision": "Decision here"}}

You will be penalized if your output cannot be parsed correctly.
"""
        pydantic_parser = PydanticOutputParser(pydantic_object=ApprovalDecision)
        prompt_template = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["position", "prompt", "criteria"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt_template | self.llm | pydantic_parser
        for _ in range(3):
            try:
                completion = chain.invoke({"position": self.position, "prompt": prompt, "criteria": criteria})
                # Validate the output before returning
                if completion.decision is not None:
                    return completion.decision
                # else:
                #     print("Validation failed: Missing required fields in completion")
                #     print("Raw output:", completion)
            except Exception as e:
                print("Exception occurred:", e)
                continue
        else:
            raise Exception("Failed to parse output after 3 attempts")


In [205]:
class Advisors(BaseModel):
    """Details of advisors generated by the leader agent."""
    positions: List[str] = Field(..., description="Positions of the advisors")
    core_principles: List[List[str]] = Field(..., description="Core principles of the advisors")

class RouteDecision(BaseModel):
    """Decision on the next advisor to process."""
    next: str = Field(..., description="The next advisor to process")

class UpdatedPrompt(BaseModel):
    """Updated prompt based on feedback from the advisor agent."""
    updated_prompt: str = Field(..., description="Updated prompt based on feedback from the advisor agent")

class LeaderAgent:
    """
    LeaderAgent class defining an agent that generates and communicates with advisor agents to help optimise prompts.
    """

    def __init__(self, base_prompt: str, criteria: str = None, advisors: List[AdvisorAgent] = None, llm = ChatOpenAI(temperature=0.0, model="gpt-4o")):
        self.base_prompt = base_prompt
        self.criteria = criteria
        self.system_message = SystemMessage(content=f"""You are an experienced: Head AI Engineer. Your core principles are:
- Always adhere to strict guidelines
- Always strive to achieve success criteria
- Always listen to feedback
- Always consider the limitations of AI models""")
        self.llm = llm
        self.advisors = advisors
        self.iterations = 0
    
    def run_approval(self, prompt: str) -> List[bool]:
        """
        Run the approval process for the prompt. Run concurrently
        """
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = [executor.submit(advisor.approval, prompt, self.criteria) for advisor in self.advisors]
            results = [future.result() for future in concurrent.futures.as_completed(futures)]
        return results
    
    def leader_decision(self, state: dict) -> RouteDecision:
        """
        LeaderAgent to decide the next advisor or to finish.
        """
        self.iterations += 1
        approval_results = self.run_approval(state["prompt"])
        print("Approval results:", approval_results)
        if all(approval_results) or (self.iterations > 10):
            return {"next": "FINISH", "prompt": state["prompt"], "messages": state["messages"]}
        else:
            # Only ask for advise from advisors that disapproved the prompt
            disapproved_advisors = [advisor for i, advisor in enumerate(self.advisors) if not approval_results[i]]
            disapproved_advisors_details = {advisor.position: advisor.core_principles for advisor in self.advisors}
            options = [advisor.position for advisor in disapproved_advisors]
            # shuffle the options to avoid positional bias
            random.shuffle(options)
            # options = ["FINISH"] + positions
            template = """```{prompt}```

Your task is to review the prompt above and decide the next advisor to provide feedback.

The success criteria for the updated prompt are as follows:
{criteria}
You must use this information to inform your decision.

The details of all advisors are their core principles are as follows: 
{advisor_roles}

Select one of the below advisors that disapproved the prompt to provide feedback on how to improve it: 
{options}

Think carefully about which aspects of the prompt need improvement and which advisor would be best suited to provide feedback on those aspects.
If you think multiple aspects of the prompt need improvement, select the most suitable advisor to provide feedback on the most critical aspect of the prompt.

{format_instructions}

Output ONLY the name of the next advisor. DO NOT output any additional information.
Example JSON ouput: {{"next": "Advisor name here"}}

You will be penalized if your output cannot be parsed correctly."""
            pydantic_parser = PydanticOutputParser(pydantic_object=RouteDecision)
            prompt_template = PromptTemplate(
                system_message=self.system_message,
                template=template,
                input_variables=["prompt", "criteria", "advisor_roles", "options"],
                partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
            )
            chain = prompt_template | self.llm | pydantic_parser
            for _ in range(3):
                try:
                    output = chain.invoke({"prompt": state["prompt"], "criteria": self.criteria, "advisor_roles": str(disapproved_advisors_details), "options": str(options)})
                    break
                except Exception as e:
                    print("Exception occurred:", e)
                    continue
            # self.updates += 1
            # if output.next == "FINISH" or self.updates == 5:
            #     return {"next": "FINISH", "prompt": state["prompt"], "messages": state["messages"]}
            # else:
                # return {"next": output.next, "prompt": state["prompt"], "messages": state["messages"]}
            return {"next": output.next, "prompt": state["prompt"]}

    def update_prompt(self, prompt: str, feedback: str) -> str:
        """
        Updates the prompt with the feedback from the advisor agent.
        """
        template = """```{prompt}```

Your task is to make updates to the prompt above by implementing the feedback below. You must action the feedback keeping in mind your core principles.
```{feedback}```

The success criteria for the updated prompt are as follows:
{criteria}
You will be penalized if the updated prompt does not meet this criteria.

Below are strict guidelines that you MUST follow if making changes to the prompt:
- DO NOT modify existing restrictions.
- DO NOT modify or remove negations.
- DO NOT add, modify or remove placeholders denoted by curly braces. IT IS ESSENTIAL PLACEHOLDERS REMAIN IN PLACE.
- ALWAYS treat placeholders as the actual content.
You will be penalized if you do not follow these guidelines.
As a reminder here is the original prompt with placeholders: 
```{base_prompt}```
The placeholders in the prompt above MUST match the placeholders in your updated prompt. 

Your update process should be as follows:
1. Read the prompt and feedback carefully.
2. Review the discussion history to ensure you do not repeat any mistakes.
3. Implement the feedback provided.
4. Ensure that your updated prompt instructs the model as expected.
5. Ensure that your update follows the strict guidelines provided above. DO NOT REMOVE PLACEHOLDERS.
6. Submit your updated prompt.

{format_instructions}

Output ONLY your updated prompt. DO NOT output any additional information.
Example JSON output: {{"updated_prompt": "Your updated prompt here"}}

You will be penalized if your output cannot be parsed correctly."""
        pydantic_parser = PydanticOutputParser(pydantic_object=UpdatedPrompt)
        prompt_template = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["prompt", "feedback", "base_prompt", "criteria"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt_template | self.llm | pydantic_parser
        for _ in range(3):
            try:
                output = chain.invoke({"prompt": prompt, "feedback": feedback, "base_prompt": self.base_prompt,  "criteria": self.criteria})
                if output.updated_prompt:
                    return output.updated_prompt
                # else:
                    # print("Validation failed: Missing required fields in completion")
                    # print("Raw output:", output)
            except Exception as e:
                print("Exception occurred:", e)
                continue

    def construct_advisor_graph(self):
        """
        Constructs a graph of advisor agents based on their roles and functions.
        """

        def agent_node(state, agent, position):
            try:
                result = agent.review_prompt(state["prompt"], self.criteria)
                feedback = result.feedback
                updated_prompt = self.update_prompt(state["prompt"], feedback)
                return {
                    "messages": state["messages"] + [
                        HumanMessage(content=f"Feedback: {feedback}", name=position),
                        AIMessage(content=f"Updated Prompt: {updated_prompt}", name="Leader")
                    ],
                    "prompt": updated_prompt,
                }
            except Exception as e:
                # Log the error and return to leader with the most recent prompt
                print(f"Parsing failed for {position}: {e}")
                return {
                    "messages": state["messages"] + [
                        HumanMessage(content=f"Error: Parsing failed for {position} - {e}", name=position),
                        AIMessage(content=f"Updated Prompt: {updated_prompt}", name="Leader")
                    ],
                    "prompt": state["prompt"],
                    "next": "leader"
                }
        
        # The agent state is the input to each node in the graph
        class AgentState(TypedDict):
            messages: Sequence[BaseMessage]
            next: str
            prompt: str

        workflow = StateGraph(AgentState)
        for advisor in self.advisors:
            # Create a node for each advisor agent
            node = functools.partial(agent_node, agent=advisor, position=advisor.position)
            workflow.add_node(advisor.position, node)
        workflow.add_node("leader", self.leader_decision)

        members = [advisor.position for advisor in self.advisors]
        for member in members:
            # We want our advisors to ALWAYS "report back" to the leader when done
            workflow.add_edge(member, "leader")
        # The leader populates the "next" field in the graph state with routes to a node or finishes
        conditional_map = {k: k for k in members}
        conditional_map["FINISH"] = END
        workflow.add_conditional_edges("leader", lambda x: x["next"], conditional_map)
        workflow.set_entry_point("leader")

        memory = SqliteSaver.from_conn_string(":memory:")
        graph = workflow.compile(checkpointer=memory)

        return graph
    
    def optimise_prompt(self):
        """
        Optimises a prompt by invoking a graph of advisor agents.
        """
        # Initial state
        initial_state = {
            "messages": [HumanMessage(content=f"Base Prompt: {self.base_prompt}", name="Leader")],
            "prompt": self.base_prompt,
            "next": "leader"
        }

        # Construct the graph
        graph = self.construct_advisor_graph()
        # display(Image(graph.get_graph().draw_mermaid_png()))

        n = random.randint(0, 1000)
        config = {
            "configurable": {"thread_id": n},
            "recursion_limit": 50,
            }    

        # Run the graph
        for s in graph.stream(
            initial_state,
            config,
            stream_mode="values",
            ):
            if "__end__" not in s:
                # print most recent message in state
                if len(s["messages"]) > 1:
                    s["messages"][-2].pretty_print()
                    s["messages"][-1].pretty_print()
                continue
        
        # if not os.path.exists("prompt_history_authoritarian.json"):
        #     with open("prompt_history_authoritarian.json", "w") as f:
        #         json.dump([], f)
        
        # with open("prompt_history_authoritarian.json", "r") as f:
        #     data = json.load(f)
        #     data.append(s)
            
        # with open("prompt_history_authoritarian.json", "w") as f:
        #     json.dump(data, f, indent=4)

        return s


In [206]:
# llm = ChatAnthropic(temperature=1.0, model="claude-3-haiku-20240307")
llm = ChatOpenAI(temperature=0.0, model="gpt-4o")

In [207]:
# Prompt design team members
style_and_structure_principles = CorePrinciples([
    "Always structure prompts logically, for example COSTAR (Context, Objective, Steps, Task, Additional Information, Result)",
    "Always use a style and tone in prompts that is appropriate for the task",
    "Always consider the complexity of the task when designing prompts",
])
style_and_structure_expert = AdvisorAgent("Style and Structure Expert", style_and_structure_principles, llm)

conciseness_and_clarity_principles = CorePrinciples([
    "Always write clear and concise prompts",
    "Always use simple and direct language in prompts",
    "Always consider the task criteria when formatting and structuring prompts",
])
conciseness_and_clarity_expert = AdvisorAgent("Conciseness and Clarity Expert", conciseness_and_clarity_principles, llm)

contextual_relevance_principles = CorePrinciples([
    "Always provide context to help the model understand the task",
    "Always consider the context in which prompts will be used",
    "Always ensure personas and roles in prompts are optimal to the context of the task",
])
contextual_relevance_expert = AdvisorAgent("Contextual Relevance Expert", contextual_relevance_principles, llm)

task_alignment_principles = CorePrinciples([
    "Always ensure that prompts align with the task criteria",
    "Always tailor instructions to the task to guide the model",
])
task_alignment_expert = AdvisorAgent("Task Alignment Expert", task_alignment_principles, llm)

example_demonstration_principal = CorePrinciples([
    "Always provide examples to help the model understand the task",
    "Always ensure examples are relevant and clear",
    "Always demonstrate the expected output of the model",
])
example_demonstration_expert = AdvisorAgent("Example Demonstration Expert", example_demonstration_principal, llm)

avoiding_bias_principles = CorePrinciples([
    "Always avoid bias in prompts",
    "Always consider the ethical implications of prompts",
])
avoiding_bias_expert = AdvisorAgent("Avoiding Bias Expert", avoiding_bias_principles)

incremental_prompting_principles = CorePrinciples([
    "Always provide clear step-by-step instructions to guide the model",
    "Always consider the complexity of the task when providing instructions",
])
incremental_prompting_expert = AdvisorAgent("Incremental Prompting Expert", incremental_prompting_principles, llm)

programming_logic_principles = CorePrinciples([
    "Only implement programming logic if you think it is beneficial for the task",
    "Structure prompts logically, similar to programming logic",
    "Implement programming logic including loops, conditionals, and functions and even pseudo-code",
])
programming_logic_expert = AdvisorAgent("Programming Logic Expert", programming_logic_principles, llm)

In [208]:
# Domain team members
mathematics_principles = CorePrinciples([
    "Always follow good mathematical practices",
    "Always use mathetical operators correctly",
    "Always consider the mathematical principles relevant to the task",
])
mathematician = AdvisorAgent("Mathematician", mathematics_principles, llm)

word_problem_solving_principles = CorePrinciples([
    "Always pay attention to the keywords in problems",
    "Always approach problems systematically",
    "Always consider multiple approaches to solving problems",
])
word_problem_solver = AdvisorAgent("Word Problem Solver", word_problem_solving_principles, llm)


In [209]:
base_prompt = "{content}\nPlease output your answer at the end as ##<answer>."
criteria = """- Correctly instructs the LLM to solve a simple maths word problem. 
- Correctly outputs the answer at the end as ##<answer> with no spaces or units."""

leader_agent = LeaderAgent(
    base_prompt=base_prompt,
    criteria=criteria,
    advisors=[
        style_and_structure_expert,
        conciseness_and_clarity_expert,
        contextual_relevance_expert,
        task_alignment_expert,
        example_demonstration_expert,
        incremental_prompting_expert,
        mathematician,
        word_problem_solver,
    ],
    llm = llm
)
for advisor in leader_agent.advisors:
    print("Position: ", advisor.position + "\nCore Principles:\n", advisor.core_principles)


Position:  Style and Structure Expert
Core Principles:
 - Always structure prompts logically, for example COSTAR (Context, Objective, Steps, Task, Additional Information, Result)
- Always use a style and tone in prompts that is appropriate for the task
- Always consider the complexity of the task when designing prompts
Position:  Conciseness and Clarity Expert
Core Principles:
 - Always write clear and concise prompts
- Always use simple and direct language in prompts
- Always consider the task criteria when formatting and structuring prompts
Position:  Contextual Relevance Expert
Core Principles:
 - Always provide context to help the model understand the task
- Always consider the context in which prompts will be used
- Always ensure personas and roles in prompts are optimal to the context of the task
Position:  Task Alignment Expert
Core Principles:
 - Always ensure that prompts align with the task criteria
- Always tailor instructions to the task to guide the model
Position:  Exampl

In [210]:
result = leader_agent.optimise_prompt()

Approval results: [False, False, False, False, False, False, False, False]
================================ Human Message =================================
Name: Task Alignment Expert

Feedback: To improve the prompt, explicitly state the need to solve a simple math word problem and ensure the answer is formatted as ##<answer> with no spaces or units. For example: 'Solve the following math word problem and provide the answer at the end in the format ##<answer> with no spaces or units.' This clarifies the task and expected output format.
================================== Ai Message ==================================
Name: Leader

Updated Prompt: {content}
Solve the following math word problem and provide the answer at the end in the format ##<answer> with no spaces or units.
Please output your answer at the end as ##<answer>.
Approval results: [False, False, False, False, False, False, False, False]
================================ Human Message =================================
Name: 

In [211]:
result["prompt"]

'Solve the following simple arithmetic problem (e.g., addition) step-by-step with detailed and logical reasoning. Show each calculation step clearly and logically on a new line, numbering each step. Ensure each step is logically explained. Then, clearly separate and provide the final answer in the format ##<answer> with no spaces or units. {content} Please output your answer at the end as ##<answer>.'

### Concurrent Runs

In [ ]:
base_prompt = "Classify the grammar in the text as correct or incorrect: {content}"
criteria = "The prompt should instruct the LLM to output either 'correct' or 'incorrect' based on the grammar in the text."

leader_agent_1 = LeaderAgent(
    base_prompt=base_prompt,
    criteria=criteria,
)
for advisor in leader_agent_1.advisors:
    print("Position: ", advisor.position + "\nRole: ", advisor.role + "\nFunction: ", advisor.function + "\n")

leader_agent_2 = LeaderAgent(
    base_prompt=base_prompt,
    criteria=criteria,
)
for advisor in leader_agent_2.advisors:
    print("Position: ", advisor.position + "\nRole: ", advisor.role + "\nFunction: ", advisor.function + "\n")

leader_agent_3 = LeaderAgent(
    base_prompt=base_prompt,
    criteria=criteria,
)
for advisor in leader_agent_3.advisors:
    print("Position: ", advisor.position + "\nRole: ", advisor.role + "\nFunction: ", advisor.function + "\n")

In [ ]:
# Assuming leader_agent is already defined and initialized
def run_optimisation(agent: LeaderAgent):
    return agent.optimise_prompt()

# Run 3 concurrent instances
with concurrent.futures.ThreadPoolExecutor(max_advisors=3) as executor:
    futures = [executor.submit(run_optimisation, agent) for agent in [leader_agent_1, leader_agent_2, leader_agent_3]]
    results = [future.result() for future in concurrent.futures.as_completed(futures)]

for result in results:
    print(result)
    print("----")

class PromptMerge(BaseModel):
    """Merged prompt based on the best parts of each prompt."""
    final_prompt: str = Field(description="Result of merging prompts")

# OpenAI Agent to pull togther best parts of each result
def merge_results(results):
    """
    Agent to merge best parts of each prompt
    """
    llm = ChatOpenAI(
        temperature=1.0,
        model="gpt-4o",
    )
    system_message = """You are an experienced AI prompt engineer. Your role is to combine good prompts to create great prompts!
You have in-depth knowledge of large language models and prompt engineering best practices. Use knowledge at all times to guide your thinking."""

    template = """Your task is to merge the best parts of the prompts below to create a better prompt.
Carefully consider the strengths of each prompt and how they can be combined effectively whilst maintaining clarity and relevance.
You will be penalized if the prompt is repetitive, lacks clarity or is incoherent.
Aspects of the prompts to consider:
- Conciseness and clarity
- Contextual relevance
- Task alignment
- Example Demonstrations
- Avoiding bias
Consider aspects of good prompts beyond those listed above.
Placeholders are notated using curly braces. You must not remove placeholders or add additional placeholders.
I repeat, you must not remove placeholders or add additional placeholders.
Do not make assumptions on what the placeholders represent.

Prompts: {results}

Return only the next advisor to process or 'FINISH' in JSON format below:

{{
    "final_prompt": "Result of merging prompts",
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
    pydantic_parser = PydanticOutputParser(pydantic_object=PromptMerge)
    prompt_template = PromptTemplate(
        system_message=system_message,
        template=template,
        input_variables=["results"],
        partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
    )
    chain = prompt_template | llm | pydantic_parser
    for _ in range(3):
        try:
            output = chain.invoke({"results": results})
            break
        except Exception as e:
            print("Exception occurred:", e)
            continue
    return output.final_prompt

final_result = merge_results(results)

In [ ]:
print(final_result)